In [26]:
# tensorboard --logdir=logs

In [27]:
from __future__ import print_function

import argparse
import sys
import os
import random

import numpy as np

import keras

In [28]:
import shutil

# make or reset directory
def mk_reset_dir(directory):
    if os.path.exists(directory):
        try:
            shutil.rmtree(directory)
            os.mkdir(directory)
        except:
            print("error:", directory)
    else:
        try:
            os.mkdir(directory)
        except:
            print("error create:", directory)

In [29]:
# getting parent directory
# project_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [30]:
# setting data path data seperated by class name
training_dir = r"data/input/train"
testing_dir = r"data/input/test"
validation_dir = r"data/input/val"

# setting output directory
model_dir =  r"data/output/models/"
log_dir = r"data/output/logs"

In [32]:
# # setting data path data seperated by class name
# training_dir = "data\\input\\train"
# testing_dir = "data\\input\\test"
# validation_dir = "data\\input\\val"

# # joining path with parent path 
# training_dir = os.path.join(project_dir, training_dir)
# testing_dir = os.path.join(project_dir, testing_dir)
# validation_dir = os.path.join(project_dir, validation_dir)


# # setting output directory
# model_dir = "data\\outputs\\models"
# log_dir = "data\\outputs\\logs"

# # joining path with parent path 
# model_dir = os.path.join(project_dir, model_dir)
# log_dir = os.path.join(project_dir, log_dir)


In [33]:
# model_dir

In [34]:
# setting training parameters
norm = 255.0
rescale=1./norm
shear_range=0.2
zoom_range=0.2
horizontal_flip=True

# setting train, test, validation parameters
target_size=(224, 224)
batch_size=32
# batch_size=128
class_mode='categorical'

loss='categorical_crossentropy'
metrics=['accuracy']

epochs = 50
verbose = 1

In [35]:
# mk_reset_dir(model_dir)
# mk_reset_dir(log_dir)

In [36]:
model_file = model_dir+"weights-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"
# model_file = "weights-{epoch:02d}-val_acc-{val_acc:.2f}.hdf5"

In [37]:
from keras.preprocessing.image import ImageDataGenerator

# data generator for training
train_datagen = ImageDataGenerator(
        rescale=rescale,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip)

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

# data generator for validation
validation_datagen = ImageDataGenerator(rescale=rescale)
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

# data generator for testing
test_datagen = ImageDataGenerator(rescale=rescale)
test_generator = test_datagen.flow_from_directory(
        testing_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=class_mode)

Found 5216 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [38]:
from keras.applications.inception_v3 import InceptionV3

# importing inception model
base_model = InceptionV3(weights='imagenet', include_top=False)

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D

# setting model layers specially output layer with class number
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)


In [ ]:
from keras.models import Model

#loading model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# set all laeyrs as untrainable
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
from keras import optimizers

sgd = optimizers.Adam()
# sgd = optimizers.SGD()
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# sgd = optimizer=SGD(lr=0.0001, momentum=0.9)

In [18]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [19]:
#print layers of inception model
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

In [20]:
# train the top inception layers 
# freeze first 249 layers
# unfreeze the rest

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [17]:
# compile model with optimizer and loss
model.compile(sgd, loss=loss, metrics=metrics)

In [21]:


checkpoint = keras.callbacks.ModelCheckpoint(model_file, monitor='val_acc', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)

In [22]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None)
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None)

In [23]:
tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
# tensorboard = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, batch_size=32, write_graph=True, write_grads=False, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)

tensorboard.set_model(model) 

In [24]:
# callbacks_list = [checkpoint, tensorboard, early_stopping]
callbacks_list = [checkpoint, tensorboard]

In [25]:
# train inception model
# fine-tuning the top layers
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    steps_per_epoch = len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=callbacks_list)

c:\users\andromeda\appdata\local\programs\python\python36\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/50
163/163 [==============================] - 161s 988ms/step - loss: 0.3829 - acc: 0.8572 - val_loss: 1.7664 - val_acc: 0.6281
Epoch 2/50
163/163 [==============================] - 103s 629ms/step - loss: 0.2416 - acc: 0.9055 - val_loss: 1.1079 - val_acc: 0.6562
Epoch 3/50
163/163 [==============================] - 106s 648ms/step - loss: 0.2316 - acc: 0.9024 - val_loss: 3.2233 - val_acc: 0.6219
Epoch 4/50
163/163 [==============================] - 104s 641ms/step - loss: 0.2201 - acc: 0.9095 - val_loss: 1.6722 - val_acc: 0.6375
Epoch 5/50
163/163 [==============================] - 105s 646ms/step - loss: 0.2239 - acc: 0.9049 - val_loss: 1.9779 - val_acc: 0.6250
Epoch 6/50
163/163 [==============================] - 105s 642ms/step - loss: 0.1993 - acc: 0.9208 - val_loss: 1.4056 - val_acc: 0.6687
Epoch 7/50
163/163 [==============================] - 111s 680ms/step - loss: 0.2066 - acc: 0.9122 - val_loss: 1.6794 - val_acc: 0.6438
Epoch 8/50
163/163 [============================

KeyboardInterrupt: 

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
#Retraining

In [ ]:
# setting retraining from old model
model_path = "data/output/models/"+"modelsweights-03-val_acc-0.89-val_loss-0.28.hdf5"
# model_path = model_+"\\data\\outputs\\models\\"+"modelsweights-03-val_acc-0.89-val_loss-0.28.hdf5"
epochs=15
verbose=1

model = keras.models.load_model(model_path)

# retrain by loading last good model
history = model.fit_generator(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=callbacks_list)

In [ ]:
# testing model
model_path = "data/output/models/"+"modelsweights-03-val_acc-0.89-val_loss-0.28.hdf5"
model = keras.models.load_model(model_path)

In [ ]:
result = model.evaluate_generator(generator=test_generator, verbose=1)

In [ ]:
print("%s%.2f%s"% ("Accuracy: ", result[1]*100, "%"))
print("%s%.2f"% ("Loss: ", result[0]))

In [ ]:
import sys

# testing model
best_accuracy = 0
best_loss = 0

lowest_accuracy = 100
lowest_loss = 1000000

models_path = "data\\output\\models\\"

model_files = os.listdir(models_path)
for model_file in model_files:
    model_path = models_path+"\\"+model_file
    
    model = keras.models.load_model(model_path)
    
    result = model.evaluate_generator(generator=test_generator, verbose=1)
    
    accuracy = result[1]*100
    loss = result[0]
    
    if accuracy>best_accuracy:
        best_accuracy = accuracy
        best_loss = loss
    if loss<lowest_loss:
        lowest_accuracy = accuracy
        lowest_loss = loss
    print("-"*80)
    print(model_file + ":")
    print("-"*80)
    
    print("%s%.2f%s"% ("Accuracy: ", accuracy, "%"))
    print("%s%.2f"% ("Loss: ", loss))
    
    print("-"*80)
    
    print("%s%.2f%s"% ("Best Accuracy: ", best_accuracy, "%"))
    print("%s%.2f"% ("Best Loss: ", best_loss))
    
    print("-"*80)
    
    print("%s%.2f%s"% ("Lowest Accuracy: ", lowest_accuracy, "%"))
    print("%s%.2f"% ("Lowest Loss: ", lowest_loss))
    
    print("-"*120)
    print("-"*120)
    
print("_"*130)
print("_"*130)

print("%s%.2f%s"% ("Best Accuracy: ", best_accuracy, "%"))
print("%s%.2f"% ("Best Loss: ", best_loss))

print("_"*100)
print("_"*100)

print("%s%.2f%s"% ("Best Accuracy: ", lowest_accuracy, "%"))
print("%s%.2f"% ("Best Loss: ", lowest_loss))


In [ ]:
import re
def name_correct(name):
    return re.sub(r'[^a-zA-Z,:]', ' ', name).title()

In [ ]:
# testing model
model_path = "data/output/models/"+"modelsweights-03-val_acc-0.89-val_loss-0.28.hdf5"
model = keras.models.load_model(model_path)

In [ ]:
! pip install Pillow

In [ ]:
from IPython.display import display
from PIL import Image

preds = model.predict_generator(test_generator, verbose=1, steps=len(test_generator))
y_classes = preds.argmax(axis=-1)

In [ ]:
# ! pip install mlxtend

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

CM = confusion_matrix(test_generator.classes, y_classes)
fig, ax = plot_confusion_matrix(conf_mat=CM ,  figsize=(12,8), hide_ticks=True,cmap=plt.cm.Blues)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

In [ ]:
target_names = ['Normal', 'Pneumonia']
print(classification_report(test_generator.classes, y_classes, target_names=target_names))

In [ ]:
# Calculate Precision and Recall
tn, fp, fn, tp = CM.ravel()

precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("Recall of the model is {:.2f}".format(recall))
print("Precision of the model is {:.2f}".format(precision))

In [ ]:
label_map = (test_generator.class_indices)
label_map_rev = {v: name_correct(k) for k,v in label_map.items()}

In [ ]:
label_map_rev

In [ ]:
num_batch_t = len(validation_generator)
num = random.randint(0, num_batch_t-1)
num

In [ ]:
y_img_batch, y_class_batch = validation_generator[num] 
y_pred = np.argmax(model.predict(y_img_batch),-1)
y_true = np.argmax(y_class_batch,-1)
print(sum(y_pred==y_true)/batch_size)

In [ ]:
y_true_labels = [label_map_rev[c] for c in y_true]
y_pred_labels = [label_map_rev[c] for c in y_pred]

In [ ]:
batch_size_t = len(y_true_labels)

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')

fig, axs = plt.subplots(nrows=8, ncols=4, figsize=(9, 18),
                        dpi=100, facecolor='w', edgecolor='k',
                        subplot_kw={'xticks': [], 'yticks': []})

plt.rcParams.update({'axes.titlesize': 'small'})
plt.subplots_adjust(hspace=0.5, wspace=0.3)


for i in range(0, batch_size_t): # how many imgs will show from the 3x3 grid
    plt.subplot(8, 4, i+1)
    plt.imshow(y_img_batch[i])
    plt.xticks([])
    plt.yticks([])
    if y_true_labels[i]==y_pred_labels[i]:
        plt.title("org: " + y_true_labels[i] + "\npred: " + y_pred_labels[i])
    else:
        plt.title("org: " + y_true_labels[i] + "\npred: " + y_pred_labels[i], color='red')
        
    fig.savefig("result", dpi=100)
plt.tight_layout()
plt.show()


In [ ]:
# ranfdomly selected image prediction visualization
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# figure(num=None, figsize=(8, 6), dpi=100, facecolor='w', edgecolor='k')

fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(8, 8),
                        dpi=100, facecolor='w', edgecolor='k',
                        subplot_kw={'xticks': [], 'yticks': []})

plt.rcParams.update({'axes.titlesize': 'small'})
plt.subplots_adjust(hspace=0.5, wspace=0.3)


m = {}

for i in range(0,9): # how many imgs will show from the 3x3 grid
    num=-1
    while num in m:
        num = random.randint(0, 31)
    m[num]=1
    plt.subplot(3, 3, i+1)
    plt.imshow(y_img_batch[num])
    plt.xticks([])
    plt.yticks([])
    if y_true_labels[num]==y_pred_labels[num]:
        plt.title("org: " + y_true_labels[num] + "\npred: " + y_pred_labels[num])
    else:
        plt.title("org: " + y_true_labels[num] + "\npred: " + y_pred_labels[num], color='red')
        
    fig.savefig("sample", dpi=100)
plt.tight_layout()
plt.show()
